In [1]:
import sys
import numpy as np
import networkx as nx
import gmsh

In [10]:
def create_pentagon(radius=1.0, clf=0.5, visualize=False):

    gmsh.initialize()

    # Create a new model
    gmsh.model.add("pentagon")

    # Define pentagon vertices (centered at origin)
    num_sides = 5
    angle_increment = 2 * np.pi/num_sides
    points = []

    for i in np.arange(num_sides):
        angle = i * angle_increment
        x     = radius * np.cos(angle)
        y     = radius * np.sin(angle)
        points.append(gmsh.model.geo.addPoint(x, y, 0, 1.0))

    # Create pentagon by adding lines between consecutive vertices
    lines = []
    for i in np.arange(num_sides):
        lines.append(gmsh.model.geo.addLine(points[i], points[(i + 1) % num_sides]))

    # Create a curve loop and a plane surface
    curve_loop = gmsh.model.geo.addCurveLoop(lines)
    surface    = gmsh.model.geo.addPlaneSurface([curve_loop])

    # Synchronize the internal CAD representation with the Gmsh model
    gmsh.model.geo.synchronize()
    
    # Control the mesh resolution by setting the target characteristic length
    gmsh.option.setNumber("Mesh.CharacteristicLengthFactor", clf) # Adjust for triangle count control

    # Generate the 2D mesh
    gmsh.model.mesh.generate(2)

    # Save the mesh to a file
    gmsh.write("reg_pentagon.msh")

    # Visualize the result
    if visualize and ('-nopopup' not in sys.argv):
        gmsh.fltk.run()

    # Finalize the gmsh session
    gmsh.finalize()

In [18]:
# Create pentagon triangulation
radius = 1.0
clf = 0.5
visualize = True

create_pentagon(radius, clf, visualize)

In [40]:
# Initialize gmsh
gmsh.initialize()

# Open mesh file
gmsh.open("reg_pentagon.msh")

# Extract node data
# node_tags: list of node IDs (integers)
# node_coords: list of node coordinates (flattened list: x1, y1, z1, x2, y2, z2, ...)
node_tags, node_coords, _ = gmsh.model.mesh.getNodes()

# Extract node coordinates into a list of tuples (x, y)
# The z-coordinates will be 0 for a 2D mesh
nodes = [(node_coords[i], node_coords[i + 1]) \
         for i in np.arange(0, len(node_coords), 3)]
# for i, ncoord in enumerate(nodes):
#     print(f"Node {i} coords: {ncoord}")

# Extract elements (triangular elements)
# element_types: list of element types (e.g., 2 is for lines, 3 is for triangles)
# element_tags: list of element IDs (integers)
# element_node_tags: connectivity of nodes for each element (flattened list)
elem_types, elem_tags, elem_node_tags = gmsh.model.mesh.getElements()

# We are interested in triangles, which have type 2 (for triangles in 2D)
# element_node_tags is a list of node indices defining the triangular elements
triangle_nodes = elem_node_tags[1]  # Assuming the second element type corresponds to triangles

# Convert the flattened node list into tuples of three nodes (triangles)
triangles = [(triangle_nodes[i], triangle_nodes[i+1], triangle_nodes[i+2]) 
              for i in range(0, len(triangle_nodes), 3)]

# Get coordinates for each triangle
num_triangles = len(triangles)
triangle_coords = []
for i in np.arange(num_triangles):
    tri_coords = []
    for node in triangles[i]:
        tri_coords.append(nodes[node-1])
    triangle_coords.append(tri_coords)

for i, tri_coords in enumerate(triangle_coords):
    print(f"Triangle {i} has coords: {tri_coords}")

def get_tri_coords(tri):
    coord1 = np.array(nodes[tri[0]])
    coord2 = np.array(nodes[tri[1]])
    coord3 = np.array(nodes[tri[2]])
    return np.array([coord1, coord2, coord3])

def get_tri_edges(tri):
    tri_coords = get_tri_coords(tri) # Array of arrays [[x1, y1], ..., [x3, y3]]
    pass

gmsh.finalize()

Triangle 0 has coords: [(np.float64(0.1852946268515169), np.float64(0.5433801289954426)), (np.float64(-0.1451326189687541), np.float64(0.44667227211061)), (np.float64(0.05724581434730305), np.float64(0.1470990987342438))]
Triangle 1 has coords: [(np.float64(0.3987739700317781), np.float64(0.2754914279899027)), (np.float64(0.1852946268515169), np.float64(0.5433801289954426)), (np.float64(0.05724581434730305), np.float64(0.1470990987342438))]
Triangle 2 has coords: [(np.float64(-0.06366100187403967), np.float64(0.8299660949612445)), (np.float64(-0.4363389981239932), np.float64(0.7088756736270214)), (np.float64(-0.1451326189687541), np.float64(0.44667227211061))]
Triangle 3 has coords: [(np.float64(-0.8090169943749473), np.float64(0.1959284174317935)), (np.float64(-0.8090169943749475), np.float64(-0.1959284174298773)), (np.float64(-0.5046309306673797), np.float64(0.01157897100203823))]
Triangle 4 has coords: [(np.float64(-0.4363389981259274), np.float64(-0.7088756736263929)), (np.float64(